# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [4]:
problem = '''We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better. 

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions. 

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. '''

## 2. Generate the mathematical model

In [19]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [20]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Radiotherapy Treatment Planning

**Parameters:**

*  `n`: Number of patients (n=17)
*  `C`: Total proton fraction capacity (C=100)
*  `BEDi(j, 15-j)`: BED score for patient `i` receiving `j` proton fractions and `15-j` photon fractions (obtained from "ProblemData.csv").

**Decision Variables:**

*  `xi,j`: Binary variable equal to 1 if patient `i` receives `j` proton fractions, and 0 otherwise.

**Objective Function:**

Maximize the total BED score for all patients:

```
Maximize  ∑(i=1 to n) ∑(j=0 to 15) BEDi(j, 15-j) * xi,j 
```

**Constraints:**

1. **Each patient receives exactly 15 fractions:**
   ```
   ∑(j=0 to 15) xi,j = 1  for all i = 1, ..., n
   ```
2. **Total proton fraction capacity:**
   ```
   ∑(i=1 to n) ∑(j=0 to 15) j * xi,j  ≤ C 
   ```
3. **Binary variables:**
   ```
   xi,j ∈ {0, 1}  for all i = 1, ..., n and j = 0, ..., 15
   ```

This model is an integer linear programming model because the objective function and constraints are linear, and the decision variables are binary. By solving this model, we can determine the optimal number of proton fractions for each patient to maximize the total BED score, given the limited proton therapy capacity. 


## 3. Generate the pyomo code

In [21]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [22]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from "ProblemData.csv")
BED = {
    (1, 0, 15): 70,
    (1, 1, 14): 72,
    # ... Add BED scores for all patients and fraction combinations
    (17, 14, 1): 88,
    (17, 15, 0): 90,
}
n = 17  # Number of patients
C = 100  # Total proton fraction capacity

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)  # Set of patients
model.J = pyo.RangeSet(0, 15)  # Set of proton fractions

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(BED[i, j, 15-j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def constraint_fractions_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.constraint_fractions = pyo.Constraint(model.I, rule=constraint_fractions_rule)

def constraint_capacity_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C
model.constraint_capacity = pyo.Constraint(rule=constraint_capacity_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
for i in model.I:
    for j in model.J:
        if pyo.value(model.x[i, j]) == 1:
            print(f"Patient {i} receives {j} proton fractions and {15-j} photon fractions.")
```

**Explanation:**

1. **Sample Data:**  The `BED` dictionary holds the BED scores. This is sample data and should be replaced with actual data from "ProblemData.csv".
2. **Sets:** `model.I` represents the set of patients, and `model.J` represents the set of possible proton fractions (0 to 15).
3. **Variables:** `model.x` is a binary variable, where `model.x[i, j] = 1` means patient `i` receives `j` proton fractions.
4. **Objective Function:** The `objective_rule` defines the objective function, which maximizes the total BED score across all patients.
5. **Constraints:**
    - `constraint_fractions_rule` ensures that each patient receives exactly 15 fractions.
    - `constraint_capacity_rule` enforces the total proton fraction capacity limit.
6. **Solve:** A solver ('glpk' in this case) is used to solve the model.
7. **Print Results:** The optimal solution is printed, indicating the number of proton fractions each patient should receive.

This code provides a Pyomo implementation for the provided mathematical model, enabling you to determine the optimal proton fraction allocation for each patient to maximize the total BED score while respecting the capacity constraints.


## 4. Input problem data and try running the generated code

In [23]:
import pyomo.environ as pyo

# Sample Data (Replace with actual data from "ProblemData.csv")
# BED = {
#     (1, 0, 15): 70,
#     (1, 1, 14): 72,
#     # ... Add BED scores for all patients and fraction combinations
#     (17, 14, 1): 88,
#     (17, 15, 0): 90,
# }

#LOADING THE DATA DONE BY HUMAN
df = pd.read_csv('ProblemData.csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j, 15-j)] = dose
#END


n = 17  # Number of patients
C = 100  # Total proton fraction capacity

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)  # Set of patients
model.J = pyo.RangeSet(0, 15)  # Set of proton fractions

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(BED[i, j, 15-j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def constraint_fractions_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.constraint_fractions = pyo.Constraint(model.I, rule=constraint_fractions_rule)

def constraint_capacity_rule(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C
model.constraint_capacity = pyo.Constraint(rule=constraint_capacity_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
for i in model.I:
    for j in model.J:
        if pyo.value(model.x[i, j]) == 1:
            print(f"Patient {i} receives {j} proton fractions and {15-j} photon fractions.")


Problem: 
- Name: unknown
  Lower bound: 8.24
  Upper bound: 8.24
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 272
  Number of nonzeros: 527
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 45
      Number of created subproblems: 45
  Error rc: 0
  Time: 0.04017519950866699
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Patient 1 receives 8 proton fractions and 7 photon fractions.
Patient 2 receives 8 proton fractions and 7 photon fractions.
Patient 3 receives 3 proton fractions and 12 photon fractions.
Patient 4 receives 0 proton fractions and 15 photon fractions.
Patient 5 receives 5 proton fractions and 10 photon fractions.
Patient 6 receives 0 proton fractions and 15 photon fractions.
Patient 7 receives 4 proton fractions and 11 photon fractions.
Patient 8 receives 15 proton fractions and 0 photon fractions.
Patient 9 receives 

In [24]:
model.obj()

8.239999999999998

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [25]:
print(response.text)

## Mathematical Optimization Model for Radiotherapy Treatment Planning

**Parameters:**

*  `n`: Number of patients (n=17)
*  `C`: Total proton fraction capacity (C=100)
*  `BEDi(j, 15-j)`: BED score for patient `i` receiving `j` proton fractions and `15-j` photon fractions (obtained from "ProblemData.csv").

**Decision Variables:**

*  `xi,j`: Binary variable equal to 1 if patient `i` receives `j` proton fractions, and 0 otherwise.

**Objective Function:**

Maximize the total BED score for all patients:

```
Maximize  ∑(i=1 to n) ∑(j=0 to 15) BEDi(j, 15-j) * xi,j 
```

**Constraints:**

1. **Each patient receives exactly 15 fractions:**
   ```
   ∑(j=0 to 15) xi,j = 1  for all i = 1, ..., n
   ```
2. **Total proton fraction capacity:**
   ```
   ∑(i=1 to n) ∑(j=0 to 15) j * xi,j  ≤ C 
   ```
3. **Binary variables:**
   ```
   xi,j ∈ {0, 1}  for all i = 1, ..., n and j = 0, ..., 15
   ```

This model is an integer linear programming model because the objective function and constraints 

In [26]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from "ProblemData.csv")
BED = {
    (1, 0, 15): 70,
    (1, 1, 14): 72,
    # ... Add BED scores for all patients and fraction combinations
    (17, 14, 1): 88,
    (17, 15, 0): 90,
}
n = 17  # Number of patients
C = 100  # Total proton fraction capacity

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)  # Set of patients
model.J = pyo.RangeSet(0, 15)  # Set of proton fractions

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(BED[i, j, 15-j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def constraint_fractions_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.constraint_fractions = pyo.Constraint(model.I, rule=constraint_fractions_rule)

def constraint_capacity_rule(model):
    return sum(j * 